In [1]:
import sys
sys.path.append("/home/missinguser/CSE/single-stage-opt/hybrid_tokamak/laptop/")
import latexplot
latexplot.set_cmap(4)
import os
from typing import Literal
import numpy as np
import matplotlib.pyplot as plt
import simsopt
from simsopt import mhd
from simsopt import geo
import json
from scipy.spatial.distance import cdist
from scipy.stats import linregress
from scipy.io import netcdf
import scipy.ndimage.filters
import warnings
from pathlib import Path
import asyncio
from concurrent.futures import ThreadPoolExecutor
from simsopt.configs import get_QUASR_data
from simsopt.mhd.vmec_diagnostics import vmec_compute_geometry

sys.path.append("/home/missinguser/CSE/single-stage-opt/")
sys.path.append("/home/missinguser/CSE/regcoil")
from replicate_lgradb.find_single_l import find_regcoil_distance


MAX_SVD_VALS = 40
SINGLE_STAGE_PATH = Path.home() / "single-stage-opt/"


def coil_surf_distance(curves, lcfs) -> np.ndarray:
    pointcloud1 = lcfs.gamma().reshape((-1, 3))
    distances = [np.min(cdist(pointcloud1, c.gamma()), axis=0) for c in curves]
    return np.array(distances).T


def compute_coil_surf_dist(simsopt_filename):
    surfaces, coils = simsopt.load(simsopt_filename)
    lcfs = surfaces[-1].to_RZFourier()

    curves = [c.curve for c in coils]
    return coil_surf_distance(curves, lcfs)


def fit_exponential_rate(sequence, title):
    x = np.linspace(0, 1, len(sequence))
    # plt.semilogy(x, sequence)
    fit = np.polyfit(x, np.log(sequence), 1)
    # plt.semilogy(x, np.exp(np.polyval(fit, x)))
    # plt.title(title)
    # plt.show()
    return fit


def compare_bdistrib(simsopt_name):
    filepath = f"{SINGLE_STAGE_PATH}/replicate_lgradb/tmp/dist05/bdistrib_out.{simsopt_name.replace('.json','.nc')}"
    # Some fits of the exponential decay, take a look at compareSingularValuesPlot.py and bdistrib_util.py

    bdistrib_variables = [
        "Bnormal_from_const_v_coils_inductance",
        "Bnormal_from_const_v_coils_transfer",
        # "svd_s_transferMatrix",
        # "svd_s_inductance_plasma_middle",
    ]
    fit_types = [
        "log_linear",
        #  "value",
        #  "windowed_upper_bound"
    ]
    fits = {}
    with netcdf.netcdf_file(filepath, "r", mmap=False) as f:
        for key in bdistrib_variables:
            vararray = f.variables[key][()].flatten()
            vararray = np.abs(vararray)  # [:MAX_SVD_VALS]
            for fit_type in fit_types:
                if fit_type == "value":
                    fit_val = np.max(np.log(vararray))
                elif fit_type == "log_linear":
                    # Only take the 0 element (slope) of the fit
                    fit_val = fit_exponential_rate(vararray, key)[0]
                else:
                    windowed_array = scipy.ndimage.filters.maximum_filter1d(
                        vararray, size=16
                    )
                    fit_val = fit_exponential_rate(windowed_array, key)[0]
                fits[key + fit_type] = fit_val
    print(list(fits.keys()))
    return fits


def vs_plot(x_data, y_data, labels=None):
    x_vals, x_label = x_data
    y_vals, y_label = y_data
    title = x_label + " vs " + y_label

    if len(np.shape(y_vals)) >= 2:
        y_vals = np.array(y_vals).T
    elif len(np.shape(y_vals)) == 1:
        y_vals = np.reshape(y_vals, (1,) + np.shape(y_vals))

    # Linear fit
    # TODO this Fails because some values are inf!!
    for i, y in enumerate(y_vals):
        plt.scatter(x_vals, y, label=title if labels is None else labels[i])
        reg = linregress(x_vals, y)
        plt.axline(
            xy1=(0, reg.intercept),
            slope=reg.slope,
            color="k",
            label=f"Linear fit {i}: $R^2$ = {reg.rvalue:.2f}",
        )

    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    if np.min(y_vals) >= 0:
        plt.gca().set_ylim(bottom=0)
    plt.gca().set_xlim(left=0)
    plt.grid(True)
    plt.legend()


In [ ]:

regcoil_plot = True
bdistrib_plot = True


import pandas as pd
df:pd.DataFrame = pd.read_pickle("../quasr_coil_check/QUASR_db/QUASR_08072024.pkl")

# Filter df by constant number of coils
df = df[df["nc_per_hp"] * df["nfp"] == 6]
df = df.sample(n=500, replace=False, random_state=10)
print(df.columns)
print(df["ID"])


LgradB_keyed = {}
coil_surf_dist = {}

def getLgradB(vmec:mhd.Vmec):
    vmec.run()
    s = [0.75, 0.96]
    ntheta = 64
    nphi = 64
    theta = np.linspace(0, 2 * np.pi, ntheta, endpoint=False)
    phi = np.linspace(0, 2 * np.pi / vmec.wout.nfp, nphi, endpoint=False)
    data = vmec_compute_geometry(vmec, s, theta, phi)

    gradB = np.array([[data.grad_B__XX, data.grad_B__YX, data.grad_B__ZX],
                      [data.grad_B__XY, data.grad_B__YY, data.grad_B__ZY],
                      [data.grad_B__XZ, data.grad_B__YZ, data.grad_B__ZZ]])
    assert np.shape(gradB)[:2] == (3, 3)
            
    return {
        "grad(B)": gradB,
        "|B|": data.modB
    }

desc_outputs = list(
    filter(lambda x: x.endswith("_output.h5"), os.listdir()))
ids = df["ID"].tolist()

vmec = mhd.Vmec(verbose=False)
# vmec.indata.mpol = 6
# vmec.indata.ntor = 5
# vmec.indata.niter_array[:3] = [1000,2500, 4000]
# vmec.indata.ns_array[:3] = [6, 13, 31]
# vmec.indata.ftol_array[:3] = [1e-5, 1e-7, 1e-10]


def get_QUASR_data_sync(idx, style='quasr-style'):
    # Your original synchronous function
    data = get_QUASR_data(idx, style)
    return idx, data[0], data[1]

async def fetch_data(idx, loop, executor):
    surfs, coils = await loop.run_in_executor(executor, get_QUASR_data_sync, idx)
    return idx, surfs, coils
    # try:
    # # Network error
    # except ValueError as e:
    #     print(f"Network error with {idx}: {e}")
    #     return idx, None, None

# loop = asyncio.get_event_loop()
# executor = ThreadPoolExecutor()
# tasks = [fetch_data(idx, loop, executor) for idx in ids]
# results = await asyncio.gather(*tasks)
results = map(get_QUASR_data_sync, ids)
results = list(filter(lambda x: x[1] is not None, results))
# loop.stop()
# executor.shutdown()

In [3]:

from joblib import Memory
location = './.cachedir'
memory = Memory(location, verbose=0)


def lgradbsc(computed, idx):
    gradB = np.array([[computed.grad_B__XX, computed.grad_B__YX, computed.grad_B__ZX],
                      [computed.grad_B__XY, computed.grad_B__YY, computed.grad_B__ZY],
                      [computed.grad_B__XZ, computed.grad_B__YZ, computed.grad_B__ZZ]])
    LgradBs = (
        np.sqrt(2)
        * computed.modB
        / np.linalg.norm(gradB, ord="fro", axis=(0, 1))
    )
    LgradBnucs = (
        np.sqrt(2)
        * computed.modB
        / np.linalg.norm(gradB, ord="nuc", axis=(0, 1))
    )
    LgradB2s = (
        np.sqrt(2)
        * computed.modB
        / np.linalg.norm(gradB, ord=2, axis=(0, 1))
    )
    return  np.array([ np.min(LgradBs), np.min(LgradB2s), np.min(LgradBnucs)])

@memory.cache(ignore=["vmec"])
def vmec_lgradbsc(vmec, idx):
    try:
        vmec.run()
        s = [0.98]
        ntheta = 64
        nphi = 64
        theta = np.linspace(0, 2 * np.pi, ntheta, endpoint=False)
        phi = np.linspace(0, 2 * np.pi / vmec.boundary.nfp, nphi, endpoint=False)
        data = vmec_compute_geometry(vmec, s, theta, phi)
        B0 = np.mean(data.modB)
        print("B0", B0)
        vmec.indata.phiedge = 5.865 * vmec.indata.phiedge / B0
        vmec.recompute_bell()
        vmec.run()
        computed = vmec_compute_geometry(vmec, s, theta, phi)
        return lgradbsc(computed, idx)
    except Exception as e:
        print(f"VMEC didn't converge! Error with {idx}: {str(e)}")
    return np.array([0.0, 0.0, 0.0])

In [ ]:
vmecs = {}
regcoil_distances = {}
for idx, surfs, coils in results:
    s : geo.SurfaceRZFourier = surfs[-1].to_RZFourier().copy()

    # If the configurations are all scaled to the same major radius instead of minor radius, the results that follow are not qualitatively different
    # scaling = 1.704 / s.minor_radius()
    # s.rc_array *= scaling
    # s.zs_array *= scaling
    scaling = 1

    vmec.boundary = s
    vmec.indata.nfp = vmec.boundary.nfp
    print(idx)
    vmecs[idx] = vmec_lgradbsc(vmec, idx)

    print(vmecs[idx])
    if np.allclose(vmecs[idx], 0): 
        regcoil_distances[ f"{idx:07d}" ] = 0
    else:
        # Must pass 
        regcoil_distances[ f"{idx:07d}" ] = find_regcoil_distance(vmec, idx)
    print("Lregcoil", regcoil_distances[ f"{idx:07d}" ])

In [ ]:
if __name__ == "__main__":
    latexplot.set_cmap(3)
    
    for idx, surfs, coils in results:
        filename =  f"{idx:07d}" 
        print("Loading", filename)
        vmec.boundary = surfs[-1]
        vmec.indata.nfp = vmec.boundary.nfp
        # vmec.indata.mpol = 6
        # vmec.indata.ntor = 5
        # vmec.run()
        # vmec.recompute_bell()
        # vmec.indata.mpol = 10
        # vmec.indata.ntor = 10
        try:
            # computed = getLgradB(vmec)
            computed = vmecs[idx]
        except simsopt._core.util.ObjectiveFailure as e:
            print(f"VMEC didn't converge! Error with {idx}: {e}")
            continue 
        except KeyError:
            # VMEC didn't converge for this configuration, so its not a valid result
            continue

        # LgradBs = computed["L_grad(B)"]
        # gradB = np.array([[computed.grad_B__XX, computed.grad_B__YX, computed.grad_B__ZX],
        #                   [computed.grad_B__XY, computed.grad_B__YY, computed.grad_B__ZY],
        #                   [computed.grad_B__XZ, computed.grad_B__YZ, computed.grad_B__ZZ]])
        # LgradBs = (
        #     np.sqrt(2)
        #     * computed.modB
        #     / np.linalg.norm(gradB, ord="fro", axis=(0, 1))
        # )
        # LgradBnucs = (
        #     np.sqrt(2)
        #     * computed.modB
        #     / np.linalg.norm(gradB, ord="nuc", axis=(0, 1))
        # )
        # LgradB2s = (
        #     np.sqrt(2)
        #     * computed.modB
        #     / np.linalg.norm(gradB, ord=2, axis=(0, 1))
        # )
        # LgradB = np.min(LgradBs)

        # REGCOIL distance
        simsopt_name = filename.replace("input.", "serial").replace(
            "_output.h5", ".json"
        )
        LgradB_keyed[simsopt_name] = vmecs[idx]
        LgradB = LgradB_keyed[simsopt_name][0]

        # The distances here were verified with the QUASR database GUI and are correct.
        # simsopt_path = f"{SINGLE_STAGE_PATH}/quasr_coil_check/QUASR_db/simsopt_serials/{simsopt_name[6:10]}/{simsopt_name}"
        coil_surf_dist[simsopt_name] = coil_surf_distance([c.curve for c in coils], surfs[-1]) #compute_coil_surf_dist(simsopt_path) 
        print(
            simsopt_name,
            "has minimum filament coil distance",
            np.min(coil_surf_dist[simsopt_name]),
        )

        # Only plot all individual equilibria for small datasets
        if LgradB < 0.05 and False:
            latexplot.set_cmap(len(coils))
            latexplot.figure()
            cross = vmec.boundary.cross_section(0, np.linspace(0, 1, 128, endpoint=True))
            plt.subplot(2, 2, 1)
            plt.plot(cross[:,0], cross[:,2], label=f"{filename}  LgradB={LgradB}")
            cross = vmec.boundary.cross_section(np.pi/2, np.linspace(0, 1, 128, endpoint=True))
            plt.plot(cross[:,1], cross[:,2], label=f"{filename}  LgradB={LgradB}")
            plt.subplot(2, 2, 2)
            # plt.imshow(LgradB_keyed[simsopt_name][-1]) 
            # plt.colorbar()
            plt.subplot(2, 2, 4)
            # plt.imshow(LgradB_keyed[simsopt_name][0]) 
            # plt.colorbar()


            # desc.plotting.plot_2d(eq, "L_grad(B)", ax=ax[0, 1], cmap="jet_r")
            # desc.plotting.plot_2d(eq, "L_grad(B)", ax=ax[0, 1], cmap="jet_r")

            if regcoil_plot:
                plt.hlines(
                    LgradB,
                    0,
                    1,
                    linestyles="dashed",
                )
                plt.hlines(
                    regcoil_distances[filename], 
                    0,
                    1,
                )
                plt.ylim(bottom=0)
                plt.legend(
                    ["$L^*_{\\nabla B}$", "Coil winding surf. dist."])

            # QUASR Filament coil distance
            plt.subplot(2, 2, 3)
            plt.plot(coil_surf_dist[simsopt_name], label="coil")
            plt.hlines(
                LgradB, 0, coil_surf_dist[simsopt_name].shape[0], linestyles="dashed"
            )
            plt.title("Filament coil distance")
            plt.legend()
            plt.ylim(bottom=0)

            plt.suptitle(filename)
            latexplot.savenshow(filename)

    #########################

    # Extract filenames and corresponding values for plotting
    latexplot.set_cmap(3)
    validlgradb_dict = {key: value for key, value in LgradB_keyed.items() if not np.allclose(value, 0)} 
    if regcoil_plot:
        validr_dict = {key: value for key, value in regcoil_distances.items() if value != 0}
        bothLgradBandRegcoil = list(set(validlgradb_dict.keys()).intersection(validr_dict.keys()))
        regcoil_vals = ([regcoil_distances[f] for f in bothLgradBandRegcoil], "REGCOIL distance")

        LgradB_vals = ([LgradB_keyed[f] for f in bothLgradBandRegcoil], "$L^*_{\\nabla B}$") 
        coil_min_vals = (
            [np.min(coil_surf_dist[f]) for f in bothLgradBandRegcoil],
            "QUASR coil distance",
        )
        latexplot.figure()
        vs_plot(regcoil_vals, LgradB_vals, ["$L^*_{\\nabla B, F}$", "$L^*_{\\nabla B, nuc}$", "$L^*_{\\nabla B, 2}$"])
        latexplot.savenshow("regcoil_vs_lgradb")
        latexplot.figure()
        vs_plot(regcoil_vals, coil_min_vals)
        latexplot.savenshow("regcoil_vs_quasr")


    filenames = list(validlgradb_dict.keys())
    LgradB_vals = ([LgradB_keyed[f] for f in filenames], "$L^*_{\\nabla B}$") 
    if bdistrib_plot:
        try:
            Bdistrib_vals = (
                [list(compare_bdistrib(f).values()) for f in filenames],
                "efficient fields seqence",
            )
        except Exception as e:
            warnings.warn(e.__str__())
            bdistrib_plot = False

    coil_min_vals = (
        [np.min(coil_surf_dist[f]) for f in filenames],
        "QUASR coil distance",
    )

    latexplot.set_cmap(3)

    #########################
    latexplot.figure()
    vs_plot(coil_min_vals, LgradB_vals, ["$L^*_{\\nabla B, F}$", "$L^*_{\\nabla B, nuc}$", "$L^*_{\\nabla B, 2}$"])
    latexplot.savenshow("lgradb_vs_quasr")

    if bdistrib_plot:
        latexplot.figure()
        vs_plot(
            coil_min_vals,
            Bdistrib_vals,
            [
                "Bnormal_from_const_v_coils_inductancelog_linear",
                # "Bnormal_from_1_over_R_field_inductancevalue",
                # "Bnormal_from_1_over_R_field_inductancewindowed_upper_bound",
                "Bnormal_from_const_v_coils_transferlog_linear",
                # "Bnormal_from_1_over_R_field_transfervalue",
                # "Bnormal_from_1_over_R_field_transferwindowed_upper_bound",
                # "svd_s_transferMatrixlog_linear",
                # "svd_s_transferMatrixvalue",
                # "svd_s_transferMatrixwindowed_upper_bound",
                # "svd_s_inductance_plasma_middlelog_linear",
                # "svd_s_inductance_plasma_middlevalue",
                # "svd_s_inductance_plasma_middlewindowed_upper_bound",
            ],
        )
        latexplot.savenshow("lgradb_vs_bdistrib")

        if regcoil_plot:
            latexplot.figure()
            vs_plot(
                regcoil_vals,
                Bdistrib_vals,
                [
                    "Bnormal_from_const_v_coils_inductancelog_linear",
                    "Bnormal_from_const_v_coils_transferlog_linear",
                ],
            )
            latexplot.savenshow("regcoil_vs_bdistrib")

    # Plot over filenames: regcoil_distances, filament coil distance, L*_{\nabla B}
    latexplot.figure()

    if regcoil_plot:
        plt.plot(bothLgradBandRegcoil, regcoil_vals[0], marker="o", label=regcoil_vals[1], linestyle="")
    plt.plot(filenames, coil_min_vals[0], marker="s", label=coil_min_vals[1], linestyle="")
    plt.plot(filenames, LgradB_vals[0], marker="^", label=LgradB_vals[1], linestyle="")
    plt.xlabel("Filenames")
    plt.ylabel("Values")
    plt.title("Comparison of different metrics")
    plt.xticks(rotation=45)
    plt.legend()
    latexplot.savenshow("correlation_all")


# Failire cases

In [ ]:
# Plot individual VMEC fail
degenerate = list(filter(lambda x: x[0] == 1301673, results))[0]
latexplot.figure()
import plot_vmec
# import importlib
# importlib.reload(plot_vmec)
plt.subplot(1, 2, 1)
plot_vmec.plot_vmec(degenerate[1][-1].to_RZFourier())
plt.title(f"QUASR 1301673")
plt.subplot(1, 2, 2)
degenerate = list(filter(lambda x: x[0] == 653942, results))[0]
plot_vmec.plot_vmec(degenerate[1][-1].to_RZFourier())
plt.title(f"QUASR 653942")

latexplot.savenshow("broken_vmec")

# Run failed cases with higher resolution

In [ ]:
vmec = mhd.Vmec(verbose=True)
idx = 923470
# for idx, vals in LgradB_keyed.items():
#   if vals[0] < 0.05:
idx, surfs, coils = get_QUASR_data_sync(int(idx))
vmec.boundary = surfs[-1]
vmec.indata.nfp = vmec.boundary.nfp
vmec_lgradbsc(vmec, idx)

vmec.run()
s = [1]
ntheta = 50
nphi = 50
theta = np.linspace(0, 2 * np.pi, ntheta, endpoint=False)
phi = np.linspace(0, 2 * np.pi / vmec.boundary.nfp, nphi, endpoint=False)
data = vmec_compute_geometry(vmec, s, theta, phi)
B0 = np.mean(data.modB)
print("B0", B0)
vmec.indata.phiedge = 5.865 * vmec.indata.phiedge / B0
vmec.recompute_bell()
vmec.run()
computed = vmec_compute_geometry(vmec, s, theta, phi)
LgradB =  lgradbsc(computed, idx)
print(LgradB )
    

In [ ]:
vmec.iota_edge()

In [24]:
plt.figure()
plt.subplot(2, 2, 1)
plt.imshow(computed.modB[-1])
plt.colorbar()
plt.subplot(2, 2, 2)


gradB = np.array([[computed.grad_B__XX, computed.grad_B__YX, computed.grad_B__ZX],
                  [computed.grad_B__XY, computed.grad_B__YY, computed.grad_B__ZY],
                  [computed.grad_B__XZ, computed.grad_B__YZ, computed.grad_B__ZZ]])
pltqty = gradB[0,0]
# pltqty = computed.modB / np.linalg.norm(gradB, ord="fro", axis=(0, 1))
plt.imshow(pltqty[0])
plt.colorbar()
plt.savefig("atest.png")

In [ ]:
for idx, val in regcoil_distances.items():
  if val == 0:
    idx, surfs, coils = get_QUASR_data_sync(int(idx))

    vmec.boundary = surfs[-1]
    vmec.indata.nfp = vmec.boundary.nfp

    regcoil_distances[ f"{idx:07d}" ] = find_regcoil_distance(vmec, idx, 1716001)
    if not np.isfinite(regcoil_distances[ f"{idx:07d}"]):
      regcoil_distances[ f"{idx:07d}" ] = 0

In [ ]:
sys.path.append("/home/missinguser/CSE/single-stage-opt/")
from plots.plot_1soec_kam.plot_boundaries import plot_surf
idx, surfs, coils = get_QUASR_data_sync(2297655)
latexplot.figure()
plot_surf(surfs[-1])
latexplot.savenshow("regcoil_fail_boundaries"+str(idx))

In [ ]:
927187
0645889